In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime
import os
import base64
import requests

# Initialize CoinGecko API
cg = CoinGeckoAPI()

# List of coins and their CoinGecko IDs
coins = {
    'BTC': 'bitcoin',
    'ETH': 'ethereum',
    'ADA': 'cardano',
    'SOL': 'solana',
    'DOT': 'polkadot',
    'AVAX': 'avalanche-2'
}

# Function to fetch historical market cap for each coin
def fetch_historical_market_cap(coin_id):
    # Fetch historical data for price and volume (last 365 days)
    market_data = cg.get_coin_market_chart_by_id(id=coin_id, vs_currency='usd', days=365)

    # Fetch circulating supply for the coin (this is constant unless there's a change)
    coin_data = cg.get_coin_by_id(id=coin_id, localization=False)
    circulating_supply = coin_data['market_data']['circulating_supply']
    
    # Extract price and volume data
    price_data = market_data['prices']
    volume_data = market_data['total_volumes']
    
    # Create DataFrame for price and volume
    df = pd.DataFrame(price_data, columns=['timestamp', 'price'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    df.drop(columns=['timestamp'], inplace=True)
    
    # Add volume and calculate market cap
    df['volume'] = [volume[1] for volume in volume_data]
    df['market_cap'] = df['price'] * circulating_supply  # Market cap = Price * Circulating Supply
    df['symbol'] = coin_id
    
    return df

# Collect data for all coins
market_cap_data = []
for symbol, coin_id in coins.items():
    df = fetch_historical_market_cap(coin_id)
    market_cap_data.append(df)

# Combine all data into a single DataFrame
combined_df = pd.concat(market_cap_data, ignore_index=True)

# Save to Excel
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"historical_market_cap_{timestamp}.xlsx"
combined_df.to_excel(filename, index=False)

# GitHub + Airtable Upload
AIRTABLE_API_KEY = os.getenv("AIRTABLE_API_KEY")
BASE_ID = "appnssPRD9yeYJJe5"
TABLE_NAME = "Database"
airtable_url = f"https://api.airtable.com/v0/{BASE_ID}/{TABLE_NAME}"
airtable_headers = {
    "Authorization": f"Bearer {AIRTABLE_API_KEY}",
    "Content-Type": "application/json"
}

response = requests.get(airtable_url, headers=airtable_headers)
response.raise_for_status()
data_airtable = response.json()

# Check for existing file
existing_records = [
    rec for rec in data_airtable['records']
    if rec['fields'].get('Name') == "Historical Market Cap Data"
]
if existing_records and 'Database Attachment' in existing_records[0]['fields']:
    record_id = existing_records[0]['id']
else:
    record_id = None

# Upload to GitHub
GITHUB_REPO = "SagarFieldElevate/DatabaseManagement"
BRANCH = "main"
UPLOAD_PATH = "uploads"
GITHUB_TOKEN = os.getenv("GH_TOKEN")

with open(filename, "rb") as f:
    content = base64.b64encode(f.read()).decode()

upload_url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{UPLOAD_PATH}/{filename}"
upload_payload = {
    "message": f"Upload historical market cap data {timestamp}",
    "content": content,
    "branch": BRANCH
}
headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json"
}
upload_resp = requests.put(upload_url, headers=headers, json=upload_payload)
if upload_resp.status_code not in [200, 201]:
    raise Exception(f"❌ GitHub upload failed: {upload_resp.status_code} - {upload_resp.text}")

raw_url = f"https://raw.githubusercontent.com/{GITHUB_REPO}/{BRANCH}/{UPLOAD_PATH}/{filename}"

# Update or create Airtable record
if record_id:
    patch_url = f"{airtable_url}/{record_id}"
    patch_payload = {
        "fields": {
            "Database Attachment": [{
                "url": raw_url,
                "filename": filename
            }]
        }
    }
    airtable_resp = requests.patch(patch_url, headers=airtable_headers, json=patch_payload)
else:
    post_payload = {
        "records": [{
            "fields": {
                "Name": "Historical Market Cap Data",
                "Database Attachment": [{
                    "url": raw_url,
                    "filename": filename
                }]
            }
        }]
    }
    airtable_resp = requests.post(airtable_url, headers=airtable_headers, json=post_payload)

if airtable_resp.status_code != 200:
    raise Exception(f"❌ Airtable upload failed: {airtable_resp.status_code} - {airtable_resp.text}")

# Delete the file from GitHub after upload
delete_url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{UPLOAD_PATH}/{filename}"
file_sha = upload_resp.json()['content']['sha']  # Get the SHA of the uploaded file
delete_payload = {
    "message": f"Delete historical market cap data {timestamp}",
    "sha": file_sha
}

delete_resp = requests.delete(delete_url, headers=headers, json=delete_payload)
if delete_resp.status_code != 200:
    raise Exception(f"❌ GitHub file deletion failed: {delete_resp.status_code} - {delete_resp.text}")

# Cleanup local file
os.remove(filename)
print("🧽 Local file and GitHub file deleted.")
